# Linear Regression


### Imports

In [ ]:
import numpy as np # numpy 모듈 import
import torch # torch를 import
import torch.nn as nn # torch.nn을 nn으로써 import
import torch.nn.functional as func # torch.nn.functional을 func으로써 import
import torch.optim as opt # torch.optim을 opt로써 import

from google.colab import drive
drive.mount('/content/drive') # -> google drive를 google.colab에 mount

# Seed 고정
torch.manual_seed(1) # 제대로된 학습을 하고 있는지 확인하기 위해서 seed를 고정한다

Mounted at /content/drive


### 1 Data loading
| x | y |
|---|---|
| 1 | 1 |
| 2 | 2 |
| 3 | 3 |



In [ ]:
# Data 수동으로 입력하기
x_train = torch.FloatTensor([[1],[2],[3]]) # rank를 2로 설정하기 위해 각각 묶어준다
y_train = torch.FloatTensor([[1],[2],[3]]) # 위의 설명과 동일

print(x_train) # x_train 출력
print(x_train.shape) # x_train의 shape 출력
print(y_train) # y_train 출력
print(y_train.shape) # y_train의 shape 출력

tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])
tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])


### 2 Hypothesis (가정) and Cost
$$ y_{hypo} = H(x) = w \cdot x + b $$
* Linear(선형)임을 가정한 경우 위 수식을 만족해야한다
* data가 3개인 경우,
  $$\begin{bmatrix} y_1 \\ y_2 \\ y_3 \end{bmatrix} =  w \cdot \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} + b $$
  + 이를 만족하는 $w$와 $b$를 구해야 한다
* Cost function으로 Mean-Square-Error를 이용한다
$$ \frac{1}{N} \sum_{n=1}^{n=N} (y_{hypo, (n)} - y_{train, (n)})^2$$

In [ ]:
# Model 초기화 (입력 dim, 출력 dim)
model = nn.Linear(1, 1) # 1차 방정식 정의

# Hypothesis 정의
y_hypo = model(x_train) # x_train을 model에 넣어 학습을 진행한 결과 값을 y_hypo(가정)에 저장
print(list(model.parameters())) # 파라미터들의 값들을 출력
print(y_hypo) # y_hypo 출력

# Cost(Mean Sqaure Error) 계산
cost = func.mse_loss(y_hypo, y_train) # cost값을 구한다
print(cost) # cost 출력

[Parameter containing:
tensor([[0.5153]], requires_grad=True), Parameter containing:
tensor([-0.4414], requires_grad=True)]
tensor([[0.0739],
        [0.5891],
        [1.1044]], grad_fn=<AddmmBackward0>)
tensor(2.1471, grad_fn=<MseLossBackward0>)


### 3 Cost Optimization
* Optimizer로 stochastic gradient descent를 이용한다

In [ ]:
# Optimizer 설정 (learning rate = 0.01로 설정)
optimizer = opt.SGD(model.parameters(), lr=0.01)

In [ ]:
# 반복
for epoch in range(1000): # 1000번 반복

  # Cost 계산 / mse_loss(가정에의한값, 참값)
  y_hypo = model(x_train)
  cost = func.mse_loss(y_hypo, y_train)

  # cost를 이용해 model update
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번 마다 중간결과 출력
  if epoch % 100 == 99:
    params = list(model.parameters())
    w = params[0].item()
    b = params[1].item()
    print('Epoch {:4d}/{} w: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
        epoch+1, 1000, w, b, cost.item()
    ))

Epoch  100/1000 w: 1.066 b: -0.150 Cost: 0.003255
Epoch  200/1000 w: 1.052 b: -0.118 Cost: 0.002011
Epoch  300/1000 w: 1.041 b: -0.093 Cost: 0.001243
Epoch  400/1000 w: 1.032 b: -0.073 Cost: 0.000768
Epoch  500/1000 w: 1.025 b: -0.057 Cost: 0.000475
Epoch  600/1000 w: 1.020 b: -0.045 Cost: 0.000293
Epoch  700/1000 w: 1.016 b: -0.035 Cost: 0.000181
Epoch  800/1000 w: 1.012 b: -0.028 Cost: 0.000112
Epoch  900/1000 w: 1.010 b: -0.022 Cost: 0.000069
Epoch 1000/1000 w: 1.008 b: -0.017 Cost: 0.000043


### 4 Multivariable Linear Regression
$$ y_{hypo} = H(\textbf{x}) = \textbf{W} \cdot \textbf{x} + b $$
* Linear(선형)임을 가정한 경우 위 수식을 만족해야한다
  $$ y_{hypo} = \begin{bmatrix} w_{1} && w_{2} && w_{3} \end{bmatrix}  \cdot \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} + \begin{bmatrix} b \end{bmatrix} $$
  + MSE cost를 최소화하는 $w$와 $b$를 구해야 한다

In [ ]:
# Numpy를 이용해 csv file을 ndarray로 가져오기
dataset = np.loadtxt(
    '/content/drive/MyDrive/data_linear_regression (1).csv',
    delimiter=',',
    dtype=np.float32)
# 순서를 random으로 섞기
np.random.shuffle(dataset)

# torch tensor로 변환
x_train = torch.FloatTensor(dataset[:,:-1]) # 데이터 set의 앞의 3개의 data를 x_train에 저장한다
y_train = torch.FloatTensor(dataset[:,[-1]]) # 데이터 set의 뒤의 1개의 data를 y_train에 저장한다 (한묶음임을 표시하기 위해 한번 묶어주고 저장)

# Linear model
model = nn.Linear(3,1) # 입력값 3개 -> 출력값 1개
print(list(model.parameters()))
# Optimizer 설정
optimizer = opt.SGD(model.parameters(), lr=0.00001)

# 반복
for epoch in range(1000): # 1000번 반
  y_hypo = model(x_train)
  cost = func.mse_loss(y_hypo, y_train)

  # model update
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번 마다 중간결과 출력
  if epoch % 100 == 99:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch+1, 1000, cost.item()))

print(y_hypo-y_train)


[Parameter containing:
tensor([[-0.1119,  0.2710, -0.5435]], requires_grad=True), Parameter containing:
tensor([0.3462], requires_grad=True)]
Epoch  100/1000 Cost: 27.945950
Epoch  200/1000 Cost: 26.145176
Epoch  300/1000 Cost: 24.495079
Epoch  400/1000 Cost: 22.982944
Epoch  500/1000 Cost: 21.597153
Epoch  600/1000 Cost: 20.327076
Epoch  700/1000 Cost: 19.162926
Epoch  800/1000 Cost: 18.095854
Epoch  900/1000 Cost: 17.117657
Epoch 1000/1000 Cost: 16.220892
tensor([[-6.4294],
        [ 5.1373],
        [ 1.0688],
        [ 2.5786],
        [-0.2428],
        [-0.7313],
        [ 3.0094],
        [-1.6001],
        [ 1.7373],
        [ 2.0861],
        [-5.1900],
        [ 2.7283],
        [ 5.7041],
        [ 3.1929],
        [-8.5924],
        [-1.0264],
        [-0.5103],
        [-3.7555],
        [-2.7923],
        [ 3.7314],
        [ 8.3312],
        [ 2.9134],
        [-4.2887],
        [-4.5285],
        [-2.3550]], grad_fn=<SubBackward0>)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 5 Assignment
* 행렬 방정식 풀기
  - 다음 행렬 방정식을 'Linear Regression'을 이용해 풀어보자
    + 적당한 learning rate를 찾아 1000 epoch 정도 계산해본다
    + 'Pseudo Inverse'를 이용한 풀이와 비교해본다
  - Hint: y = wx 꼴로 변환해본다
    + Ax=B에서는 x가 미지수이지만, y=wx에서는 w가 미지수임에 주의!
    + linear model에서 b를 없애기 위해서 nn.Linear() 사용법을 검색해보자
$$Ax=B$$
$$A = \begin{bmatrix}0 & 1 \\ 1 & 1 \\ 2 & 1 \\ 3 & 1 \end{bmatrix} $$
$$B = \begin{bmatrix}-1 \\ 0.2 \\ 0.9 \\ 2.1 \end{bmatrix} $$

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as opt

A = torch.FloatTensor([[0,1],[1,1],[2,1],[3,1]]) # A를 배열의 형태로 수동 입력
B = torch.FloatTensor([[-1],[0.2],[0.9],[2.1]]) # B를 배열의 형태로 수동 입력

# Linear model
model = nn.Linear(2, 1,bias=False) #  입력이 2개고 출력이 1개인 모델
# b값을 없애고 싶으면, model = nn.Linear(2, 1,bias=False)로 작성

# Optimizer 설정
optimizer = opt.SGD(model.parameters(), lr=0.016) # 경사하강법을 이용하여 파라미터 값 지정하기, learning rate = 0.016

# 반복
for epoch in range(1000): #1000번 반복

  # Cost 계산
  B_hypo = model(A) # 모델을 통과한 A값을 B_hypo로 지정
  cost = func.mse_loss(B_hypo, B) # B_hypo와 B 사이의 오차(cost) 구하기

  # Model update
  optimizer.zero_grad() # gradient 구하기 (미분을 통해 구한 기울기를 0으로 바꾸기)
  cost.backward() # w와 b 업데이트하기
  optimizer.step() # 역전파를 끄는 부분

  # 100번 마다 중간결과 출력
  if epoch % 100 == 99: # 10번 마다 결과 출력하기
    params = list(model.parameters()) # 모델의 파라미터값들을 list 형태로 변환
    w=params[0][0][0].item() # w값 = param 리스트에서 가장 첫번째 값 반환 [[[w,*][*,*]]]
    b=params[0][0][1].item() # b값 = param 리스트에서 가장 안쪽 두번째 값 반환 [[[*,b],[*,*]]]

    # w, b, cost값 출력
    print('Epoch {:4d}/{} w: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
       epoch+1, 1000, w, b, cost.item()
    ))

Epoch  100/1000 w: 0.830 b: -0.175 Cost: 0.057081
Epoch  200/1000 w: 0.966 b: -0.310 Cost: 0.014271
Epoch  300/1000 w: 0.993 b: -0.338 Cost: 0.012570
Epoch  400/1000 w: 0.999 b: -0.343 Cost: 0.012503
Epoch  500/1000 w: 1.000 b: -0.344 Cost: 0.012500
Epoch  600/1000 w: 1.000 b: -0.344 Cost: 0.012500
Epoch  700/1000 w: 1.000 b: -0.344 Cost: 0.012500
Epoch  800/1000 w: 1.000 b: -0.344 Cost: 0.012500
Epoch  900/1000 w: 1.000 b: -0.344 Cost: 0.012500
Epoch 1000/1000 w: 1.000 b: -0.344 Cost: 0.012500


### 저번 과제와 비교

In [ ]:
import torch

A = torch.FloatTensor([[0,1],[1,1],[2,1],[3,1]]) # A 행렬 선언
b = torch.FloatTensor([-1,0.2,0.9,2.1]) # b 행렬 선언
Tb = b.reshape(4,1) # 계산해주기 위해 b 행렬을 4,1로 reshape한 것을 Tb로 선언
AA = (A.T).matmul(A) # trans한 A와 A를 곱한 것을 AA로 선언
TAA = torch.inverse(AA) # AA의 역행렬을 구하고 그것을 TAA로 선언
AT = A.T # A의 trans를 AT로 선언
ATb = AT.matmul(Tb) # AT와 Tb의 곱을 ATb로 선언
print(TAA.matmul(ATb)) # TAA와 ATb의 곱을 프린트함으로써 결과값 출력

tensor([[ 1.0000],
        [-0.9500]])


저번 과제는 직접 수식을 이용하여 계산을 하는 방식으로 문제를 풀었지만 이번 과제는 수식을 사용하지 않고 방정식을 사용하여 오차를 줄여가는 방식으로 풀었다는 점이 차이점이다
과정은 다르지만 결국은 비슷한 결과값을 가지는 것을 확인할 수 있다